In [1]:
# ====================================================
# import
# ====================================================
import operator
from typing import Annotated, Any, Optional

from dotenv import load_dotenv
from pydantic import BaseModel, Field


In [7]:
# ====================================================
# データ構造の定義
# ====================================================

# ペルソナを表すデータモデル
class Persona(BaseModel):
    name: str = Field(description="ペルソナの名前")
    background: str = Field(description="ペルソナの持つ背景")


# ペルソナのリストを表すデータモデル
class Personas(BaseModel):
    personas: list[Persona] = Field(default_factory=list, description="ペルソナのリスト")


# インタビュー内容を表すデータモデル
class Interview(BaseModel):
    persona: Persona = Field(description="インタビュー対象のペルソナ")
    question: str = Field(description="インタビューでの質問")
    answer: str = Field(description="インタビューでの回答")


# インタビュー結果のリストを表すデータモデル
class InterviewResult(BaseModel):
    interviews: list[Interview] = Field(default_factory=list, description="インタビュー結果のリスト")


# 評価の結果を表すデータモデル
class EvaluationResult(BaseModel):
    reason: str = Field(description="判断の理由")
    is_sufficient: bool = Field(description="情報が十分かどうか")


In [ ]:
# 要件定義書AIエージェントのステート
class InterviewState(BaseModel):
    user_request: str = Field(description="ユーザーからのリクエスト")
    personas: Annotated[list[Persona], operator.add] = Field(
        default_factory=list, description="生成されたペルソナのリスト"
    )
    interviews: Annotated[list[Interview], operator.add] = Field(
        default_factory=list, description="実施されたインタビューのリスト"
    )
    requirements_doc: str = Field(default="", description="生成された要件定義")
    iteration: int = Field(default=0, description="ペルソナ生成とインタビューの反復回数")
    is_information_sufficient: bool = Field(default=False, description="情報が十分かどうか")


In [ ]:
# ====================================================
# 主要コンポーネントの実装
# ====================================================

